In [ ]:
from tamalero.utils import get_kcu
from tamalero.ReadoutBoard import ReadoutBoard
from tamalero.ETROC import ETROC
from tamalero.colors import green, red, yellow
from tamalero.LPGBT import LPGBT
import numpy as np
from datetime import datetime, timezone
import os
import pickle
from tamalero.DataFrame import DataFrame
from tamalero.FIFO import FIFO
from random import randint
import time

KCU_IP = "192.168.0.10" ## If your KCU ip is diff, modify it.

READOUTBOARD_ID = 0
READOUTBOARD_CONFIG = 'default'

ETROC_I2C_ADDRESSES = [0x63, 0x62, 0x61, 0x60]
ETROC_I2C_CHANNEL = 1
ETROC_ELINKS_MAP = {0: [0, 4, 8, 12]}
CHARGE_FC = 30 
QINJ_COUNT = 1000

TRIGGER_MASK_0 = 0xF
TRIGGER_MASK_1 = 1
TRIGGER_MASK_2 = 472

print('ETROC COSMIC RUN TEST - HARDWARE INITIALIZAATION')


kcu = get_kcu(
    KCU_IP,
    control_hub=True,
    host='localhost',
    verbose=False
)
print(green("Successfully connected to KCU."))

kcu.status() # Prints LpGBT link statuses from KCU 
fw_ver = kcu.get_firmware_version(verbose=True) #
kcu.check_clock_frequencies(verbose=True) # Verifies KCU clock stability

# Perform a simple loopback register test to confirm communication
loopback_val = 0xABCD1234
kcu.write_node("LOOPBACK.LOOPBACK", loopback_val) #
read_val = kcu.read_node("LOOPBACK.LOOPBACK").value()
if read_val == loopback_val:
    print(green(f"KCU Loopback test PASSED: Wrote 0x{loopback_val:X}, Read 0x{read_val:X}"))
else:
    print(red(f"KCU Loopback test FAILED: Wrote 0x{loopback_val:X}, Read 0x{read_val:X}"))

# ======================================================================================
# 2. INITIALIZE READOUT BOARD
# ======================================================================================
rb = ReadoutBoard(
    rb=READOUTBOARD_ID,
    kcu=kcu,
    config=READOUTBOARD_CONFIG, 
    trigger=False,     
    verbose=False,
    allow_bad_links=True
)

print(green(f"Readout Board version detected: {rb.ver}"))

# ======================================================================================
# 3. INITIALIZE FOUR ETROC
# ======================================================================================
print("\n3. Initializing ETROC chips...")
etroc_chips = []
chip_names = []
for i, addr in enumerate(ETROC_I2C_ADDRESSES):
    chip_name = f"Chip{i+1}"
    chip_names.append(chip_name)
    print(f"\nInitializing {chip_name} (I2C: 0x{addr:02X})...")

try:
    etroc = ETROC(
            rb,
            master='lpgbt',
            i2c_adr=addr,
            i2c_channel=ETROC_I2C_CHANNEL,
            elinks=ETROC_ELINKS_MAP,
            strict=False,
            verbose=True
        )
        etroc_chips.append(etroc)

        if etroc.is_connected():
            # Check key registers
            scrambler_status = etroc.rd_reg("disScrambler")
            controller_state = etroc.rd_reg("controllerState")
            pll_unlock_count = etroc.rd_reg("pllUnlockCount")
            elink_status = etroc.get_elink_status()
            
            print(green(f"✓ {chip_name} connected successfully"))
            print(f"  Controller state: {controller_state} (should be 11)")
            print(f"  PLL unlock count: {pll_unlock_count}")
            print(f"  Elink status: {elink_status}")
            
            if scrambler_status == 1:
                print(green("  ✓ Register communication verified"))
            else:
                print(red("  ✗ Register communication issue"))
        else:
            print(red(f"✗ {chip_name} not responding"))
            
except Exception as e:
    print(red(f"✗ Failed to initialize {chip_name}: {e}"))
    etroc_chips.append(None)


   

In [ ]:
df = DataFrame()
fifo = FIFO(rb)
print("\n=== Self trigger mode: Four ETROC with multiple pixels. Get data with Qinj only ===")
results = []
time.sleep(1)
fifo.reset()
time.sleep(1)
rb.reset_data_error_count()
rb.enable_etroc_readout()

test_pixels_per_chip = [
    [(randint(0, 15), randint(0, 15))],  # Chip1: 1 pixel
    [(randint(0, 15), randint(0, 15)), (randint(0, 15), randint(0, 15))],  # Chip2: 2 pixels
    [(randint(0, 15), randint(0, 15)), (randint(0, 15), randint(0, 15))],  # Chip3: 2 pixels
    [(randint(0, 15), randint(0, 15))]   # Chip4: 1 pixel
]

etroc_configs = []
for i, (etroc, chip_name) in enumerate(zip(etroc_chips, chip_names)):
    if etroc is not None and i < len(test_pixels_per_chip):
        etroc_configs.append((etroc, chip_name, test_pixels_per_chip[i]))

print("Test pixel assignments:")
for etroc, chip_name, pixels in etroc_configs:
    print(f"  {chip_name}: {pixels}")


baseline_storage = {}

for etroc, chip_name, test_pixels in etroc_configs:
    baseline_storage[chip_name] = {}
    print(f"Scanning {chip_name} to get baseline")

    for pixel_row, pixel_col in test_pixels:
        baseline, noise_width = etroc.auto_threshold_scan(
            row=pixel_row, 
            col=pixel_col, 
            broadcast=False, 
            offset='auto',  
            use=False,      
            verbose=True
        )
        baseline_storage[chip_name][(pixel_row, pixel_col)] = baseline
print(f"Baseline storage: {baseline_storage}")

for etroc, chip_name, _ in etroc_configs:
    print(f"Configuring {chip_name} ...")
    etroc.reset()
    etroc.wr_reg("singlePort", 1)  
    etroc.wr_reg("disDataReadout", 1, broadcast=True)
    etroc.wr_reg("QInjEn", 0, broadcast=True)
    etroc.wr_reg("enable_TDC", 0, broadcast=True)
    etroc.wr_reg("disTrigPath", 1, broadcast=True)
    etroc.wr_reg("workMode", 0, broadcast = True)
    etroc.wr_reg('triggerGranularity', 1)
    time.sleep(1)

for etroc, chip_name, test_pixels in etroc_configs:
    for pixel_row, pixel_col in test_pixels:
        print(f"Configuring {chip_name} Pixel ({pixel_row}, {pixel_col})")

        # Enable target pixel
        etroc.wr_reg("workMode", 0, row=pixel_row, col=pixel_col, broadcast=False)
        etroc.wr_reg("enable_TDC", 1, row=pixel_row, col=pixel_col, broadcast=False)
        etroc.wr_reg("disDataReadout", 0, row=pixel_row, col=pixel_col, broadcast=False)
        etroc.wr_reg("disTrigPath", 0, row=pixel_row, col=pixel_col, broadcast=False)
        time.sleep(1)
        baseline = baseline_storage[chip_name][(pixel_row, pixel_col)]
        applied_dac = baseline + 50
        etroc.wr_reg('DAC',applied_dac, row=pixel_row, col=pixel_col, broadcast=False)

        print(f"Applied DAC value: {etroc.rd_reg('DAC', row=pixel_row, col=pixel_col)}")
        
        etroc.wr_reg("QSel", charge - 1, row=pixel_row, col=pixel_col, broadcast=False)
        etroc.wr_reg("QInjEn", 1, row=pixel_row, col=pixel_col, broadcast=False)
        time.sleep(0.1)

rb.rerun_bitslip()  

for elink in [0,4,8,12]:
    locked = rb.etroc_locked(elink, slave=False)
    print(f"elink: {elink} locked status: {locked}")

print("\n Configuring Self-Trigger System")

rb.kcu.write_node(f"READOUT_BOARD_{rb.rb}.TRIG_ENABLE_MASK_0", TRIGGER_MASK_0)
rb.kcu.write_node(f"READOUT_BOARD_{rb.rb}.TRIG_ENABLE_MASK_1", TRIGGER_MASK_1)
rb.kcu.write_node(f"READOUT_BOARD_{rb.rb}.TRIG_ENABLE_MASK_3", TRIGGER_MASK_3)


rb.enable_etroc_trigger()
# print(f'self trigger status: {rb.self_trigger_status()}')
fifo.reset()
time.sleep(1)
Qinj_count = 1000
print(f"Sending QInj {Qinj_count}")
fifo.send_Qinj_only(count=Qinj_count)
time.sleep(2)
rb.disable_etroc_trigger()


try:
    data = fifo.pretty_read(df)
    occupancy = len(data)
    print(f"   FIFO returned {occupancy} data items")

    output_dir = "Qinj_Output"
    os.makedirs(output_dir, exist_ok=True)

    if occupancy > 0:
        print(green("   SUCCESS: Data was generated!"))
        all_test_pixels = test_pixels_chip1 + test_pixels_chip2

        test_results = {
            "test_parameters": {
                "timestamp_utc": datetime.now(timezone.utc).isoformat(),
                "all_test_pixels": all_test_pixels,
                "chip1_pixels": test_pixels_chip1,
                "chip2_pixels": test_pixels_chip2,
                "n_pixels": len(all_test_pixels),
                "charge_fC": charge,
                "Qinj_cnt": Qinj_count
            },
            "run_summary": {},
            "analysis": {},
            "chip_results": {},
            "parsed_hits": [], # Storing all hit data dictionaries
            "raw_events": data
        }
        chip_hits = {"Chip1": {}, "Chip2": {}, "Chip3": {}, "Chip4": {}}
        chip_data = {"Chip1": {}, "Chip2": {}, "Chip3": {}, "Chip4": {}}
    
        for etroc, chip_name, test_pixels in all_test_config:
            for pixel in test_pixels:
                chip_hits[chip_name][pixel]= 0
                chip_data[chip_name][pixel]= {'toa':[], "tot":[], "cal":[]} 
        
        # Parse the data
        header_count = 0
        hit_counter = 0
        filler_count = 0
        trailer_count = 0

        print("\n--- Multi pixels_Parsed Hit Data ---")
        for i, event in enumerate(data):  
            if event is None or len(event) < 2:
                continue

            data_type, event_data = event[0], event[1]
            if data_type == 'header':
                header_count += 1
            elif data_type == 'filler':
                filler_count += 1
            elif data_type == 'trailer':
                trailer_count += 1
            elif data_type == 'data':
                hit_counter += 1
                test_results["parsed_hits"].append(event_data) # Add full hit data
                # Extract values
                toa = event_data.get('toa')
                tot = event_data.get('tot')
                cal = event_data.get('cal')
                row = event_data.get('row_id')
                col = event_data.get('col_id')
                
                for etroc, chip_name, test_pixels in all_test_config:
                    if (row, col) in test_pixels:
                        chip_hits[chip_name][(row, col)] += 1
                        chip_data[chip_name][(row, col)]["toa"].append(toa)
                        chip_data[chip_name][(row, col)]["tot"].append(tot)
                        chip_data[chip_name][(row, col)]["cal"].append(cal)

                    # Print the first 20 hits to the console
                        if chip_hits[chip_name][(row, col)] <= 2: 
                            print(green(f" {chip_name} Pixel({row},{col}) Hit {chip_hits[chip_name][(row, col)]} -> ToA: {toa}, ToT: {tot}, Cal: {cal}"))

        print(f"\n--- Summary ---")
        print(f"Total events processed: {len(data)}")
        print(f"Headers found: {header_count}")
        print(f"Hits detected: {hit_counter}")
        print(f'trailer found {trailer_count}')
        print(f"Filler found {filler_count}")
        
        total_expected_hits = 0
        for etroc, chip_name, test_pixels in all_test_config:
            for pixel in test_pixels:
                pixel_row, pixel_col = pixel
                hits = chip_hits[chip_name][pixel]
                total_expected_hits += hits
                # print(f"\n ---Pixel ({pixel_row}, {pixel_col}) Results ---")

                pixel_stats = {}
                if hits > 0:
                    toa_values = chip_data[chip_name][pixel]["toa"]
                    tot_values = chip_data[chip_name][pixel]["tot"]
                    cal_values = chip_data[chip_name][pixel]["cal"]

                    if toa_values:
                        pixel_stats["toa"] = {"mean" : np.mean(toa_values),}
                    if tot_values:
                        pixel_stats["tot"] = {"mean" : np.mean(tot_values),}
                    if cal_values:
                        pixel_stats["cal"] = {"mean" : np.mean(cal_values),}
                
                test_results["chip_results"][f"{chip_name}_pixel_{pixel_row}_{pixel_col}"] = {
                    "chip": chip_name,
                    "position": pixel,
                    "hits":hits,
                    "stats": pixel_stats
                }
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        ## Not using json due to standard json can't serialize Numpy data types
        pickle_filename = f"multi_board_injection_{len(all_test_config)}_pixels_{timestamp}.pkl"
        pickle_filepath = os.path.join(output_dir, pickle_filename)
        
        try:
            with open(pickle_filepath, 'wb') as f:
                pickle.dump(test_results, f)
            print(green("Successfully saved pickle file."))
            print(green("Successfully saved results."))
        except Exception as e:
            print(f"Error saving file: {e}")
        
    else:
        print(red("FIFO returned no data. No hits detected."))
except Exception as e:
    print("Exception ", e)

# Cleanup
for etroc, chip_name, _ in all_test_config:
    print(f"Cleaning up {chip_name}...")
    for _ in range(3):
        fifo.reset()
        rb.reset_data_error_count()
        etroc.wr_reg("QInjEn", 0, broadcast=True)
        #my_etroc2.wr_reg("QInjEn", 0, row=pixel_row, col=pixel_col)
        etroc.wr_reg("disDataReadout", 1, broadcast=True)
        time.sleep(0.1)
print("Cleanup completed: Disabled QInj, re-enabled all pixels")


=== Self trigger mode: Four ETROC with multiple pixels. Get data with Qinj only ===
test_pixels_chip1 = [(9, 4)]
test_pixels_chip2 = [(12, 7), (2, 9)]
test_pixels_chip3 = [(11, 4), (4, 10)]
test_pixels_chip4 = [(15, 15)]
Scanning Chip1 to get baseline
Scanning Chip2 to get baseline
Scanning Chip3 to get baseline
Scanning Chip4 to get baseline
Baseline storage: {'Chip1': {(9, 4): 570}, 'Chip2': {(12, 7): 550, (2, 9): 561}, 'Chip3': {(11, 4): 447, (4, 10): 479}, 'Chip4': {(15, 15): 390}}
Configuring Chip1 ...
Configuring Chip2 ...
Configuring Chip3 ...
Configuring Chip4 ...
Configuring Chip2 Pixel (12, 7)
Applied DAC value: 600
Configuring Chip2 Pixel (2, 9)
Applied DAC value: 611
Configuring Chip3 Pixel (11, 4)
Applied DAC value: 497
Configuring Chip3 Pixel (4, 10)
Applied DAC value: 529
Configuring Chip4 Pixel (15, 15)
Applied DAC value: 440
elink: 0 locked status: True
elink: 4 locked status: True
elink: 8 locked status: True
elink: 12 locked status: True

 Configuring Self-Trigger S